# Busca simples

Desenvolvimento de um buscador Simples: Booleano, TF-IDF, BM25

Tópicos abordados: Indexação, Bag-of-Words, TF-IDF, BM25

Aula 1 - [Unicamp - IA368DD: Deep Learning aplicado a sistemas de busca.](https://www.cpg.feec.unicamp.br/cpg/lista/caderno_horario_show.php?id=1779)

Autor: Marcus Vinícius Borela de Castro

[Repositório no github](https://github.com/marcusborela/deep_learning_em_buscas_unicamp)

[Link para chat de apoio com WebChatGPT](https://github.com/marcusborela/deep_learning_em_buscas_unicamp/blob/main/chat/CG%20uso%20no%20buscador%20aula%201.md)

[![Open In Colab latest github version](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/marcusborela/deep_learning_em_buscas_unicamp/blob/main/code/aula1_buscador_simples.ipynb) [Open In Colab latest github version]

## Enunciado exercício

Aula 2 - Notebook: Buscador Booleano/bag-of-words e buscador com TF-IDF

1. Usar o BM25 implementado pelo pyserini para buscar queries no TREC-DL 2020
Documentação referencia: https://github.com/castorini/pyserini/blob/master/docs/experiments-msmarco-passage.md
2. Implementar um buscador booleano/bag-of-words.
3. Implementar um buscador com TF-IDF
4. Avaliar implementações 1, 2, e 3 no TREC-DL 2020 e calcular o nDCG@10
Nos itens 2 e 3:

Fazer uma implementação que suporta buscar eficientemente milhões de documentos.

Não se pode usar bibliotecas como sklearn, que já implementam o BoW e TF-IDF.


## Organizando o ambiente

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory


In [ ]:
def mostra_memoria():
  vm = virtual_memory()
  ram={}
  ram['total']=round(vm.total / 1e9,2)
  ram['available']=round(virtual_memory().available / 1e9,2)
  # ram['percent']=round(virtual_memory().percent / 1e9,2)
  ram['used']=round(virtual_memory().used / 1e9,2)
  ram['free']=round(virtual_memory().free / 1e9,2)
  ram['active']=round(virtual_memory().active / 1e9,2)
  ram['inactive']=round(virtual_memory().inactive / 1e9,2)
  ram['buffers']=round(virtual_memory().buffers / 1e9,2)
  ram['cached']=round(virtual_memory().cached/1e9 ,2)
  print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")


In [ ]:
mostra_memoria()

### Vinculando pasta do google drive para salvar dados

In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls '/content/drive'

In [ ]:
current_dir = os.getcwd()
print("Current directory:", current_dir)

### Instalações de libraries

In [ ]:
!pip install git+https://github.com/castorini/pygaggle.git

In [ ]:
!pip install pyserini

In [ ]:
!pip install faiss-cpu -q

### Baixando o repositório do pyserini para usara seus scripts

In [ ]:
path_pyserini = '/content/drive/MyDrive/treinamento/202301_IA368DD/code/pyserini'
path_pyserini_tools = path_pyserini + '/pyserini-master/anserini-tools-master'
path_pyserini_eval = path_pyserini + '/pyserini-master/pyserini/eval'

In [ ]:
if not os.path.exists(path_pyserini):
    os.makedirs(path_pyserini)
    print('pasta criada')
    !wget -q https://github.com/castorini/pyserini/archive/refs/heads/master.zip -O pyserini.zip 
    !unzip -q pyserini.zip -d  {path_pyserini}
    # Baixando tools que é um atalho para https://github.com/castorini/anserini-tools
    !wget -q https://github.com/castorini/anserini-tools/archive/refs/heads/master.zip -O anserini-tools.zip 
    !unzip -q anserini-tools.zip -d  {path_pyserini}
path_pyserini = path_pyserini + '/pyserini-master'

In [ ]:
 assert os.path.exists(path_pyserini), f"Pasta {path_pyserini} não criada!"

In [ ]:
 assert os.path.exists(path_pyserini_tools), f"Pasta {path_pyserini_tools} não criada!"

In [ ]:
 assert os.path.exists(path_pyserini_eval), f"Pasta {path_pyserini_eval} não criada!"

## Carga dos dados da TREC 2020 usando pyserini

### Obtendo dados dos documentos a partir do pyserini


[Dicas aqui](https://github.com/castorini/pyserini/blob/master/docs/experiments-msmarco-passage.md)

In [ ]:
path_data = '/content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage'

In [ ]:
%%time
if not os.path.exists(path_data):
  os.makedirs(path_data)
  print('pasta criada')
  !wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz -P {path_data}
  !tar xvfz {path_data}/collectionandqueries.tar.gz -C {path_data}
  os.remove(f'{path_data}/collectionandqueries.tar.gz')
  print("Dados carregados!")
else:
  print("Dados já existiam!")    

In [ ]:
 assert os.path.exists(path_data), f"Pasta {path_data} não criada!"

Passo anterior gera os seguintes arquivos:

* collection.tsv
* qrels.dev.small.tsv
* qrels.train.tsv
* queries.dev.small.tsv
* queries.dev.tsv
* queries.eval.small.tsv
* queries.eval.tsv
* queries.train.tsv

Next, we need to convert the MS MARCO tsv collection into Pyserini's jsonl files (which have one json object per line):

In [ ]:
%%time
if not os.path.exists(f'{path_data}/collection_jsonl'):
  !python {path_pyserini_tools}/tools/scripts/msmarco/convert_collection_to_jsonl.py \
  --collection-path {path_data}/collection.tsv \
  --output-folder {path_data}/collection_jsonl
  print("Dados carregados!")
else:
  print("Dados já existiam!")    

In [ ]:
assert os.path.exists(f'{path_data}/collection_jsonl'), f"Pasta {path_data}/collection_jsonl não criada!"

The above script should generate 9 jsonl files in collections/msmarco-passage/collection_jsonl, each with 1M lines (except for the last one, which should have 841,823 lines).

Convertendo as queries (small dev) para o formato trec para avaliações futuras

In [ ]:
if not os.path.exists(f'{path_data}/qrels.dev.small.trec'):
  !python {path_pyserini_tools}/scripts/msmarco/convert_msmarco_to_trec_qrels.py \
  --input {path_pyserini_tools}/topics-and-qrels/qrels.msmarco-passage.dev-subset.txt \
  --output {path_data}/qrels.dev.small.trec
  print("Conversão efetuada!")
else:
  print("Arquivo já existia!")


### Loading data in dicts

The 6980 queries in the development set are already stored in the repo. Let's take a peek:

In [ ]:
!head {path_pyserini_tools}/topics-and-qrels/topics.msmarco-passage.dev-subset.txt

In [ ]:
!head {path_pyserini_tools}/topics-and-qrels/topics.dl20.txt

In [ ]:
!head {path_data}/qrels.dev.small.trec

In [ ]:
!head {path_pyserini_tools}/topics-and-qrels/qrels.msmarco-passage.dev-subset.txt

In [ ]:
!head {path_pyserini_tools}/topics-and-qrels/qrels.dl20-passage.txt

Each line contains a tab-delimited (query id, query) pair. Conveniently, Pyserini already knows how to load and iterate through these pairs. We can now perform retrieval using these queries:

#### Carregando queries

##### Carregando do arquivo

In [ ]:
def ler_arquivo_query_trec20(file_path:str):
  """
  Função para ler um arquivo de queries TREC 2020 e retorná-las em um dicionário.

  Args:
    file_path (str): Caminho do arquivo de queries TREC 2020

  Returns:
    dict: Dicionário em que as chaves são os IDs das queries e os valores são os
          textos das queries correspondentes.
  """

  # Cria um dicionário vazio para armazenar as queries
  query_dict = {}

  # Abre o arquivo em modo leitura
  with open(file_path, 'r') as f:
      
      # Itera sobre as linhas do arquivo
      for line in f:

          # Separa a linha em duas partes (id e texto), considerando que são separadas por uma tabulação
          query_id, query_text = line.strip().split('\t')
          query_id = int(query_id)
          # Adiciona a query ao dicionário, usando o id como chave e o texto como valor
          query_dict[query_id] = query_text

  # Retorna o dicionário com as queries
  return query_dict

Verificando queries de todo o dev dataset (total 6980)

In [ ]:
query_dev_dict = ler_arquivo_query_trec20(f'{path_pyserini_tools}/topics-and-qrels/topics.msmarco-passage.dev-subset.txt')

In [ ]:
len(query_dev_dict),list(query_dev_dict.items())[:4]

Carregando o queries do trec20 dataset (total 200)

In [ ]:
query_trec20_dict = ler_arquivo_query_trec20(f'{path_pyserini_tools}/topics-and-qrels/topics.dl20.txt')

In [ ]:
len(query_trec20_dict),list(query_trec20_dict.items())[:4]

##### Carregando usando get_topics

In [ ]:
from pyserini.search import get_topics

In [ ]:
topics = get_topics('dl20')
print(f'{len(topics)} queries total')

In [ ]:
len(topics), list(topics.items())[0]

#### Carregando qrel (relevância por query)

##### Carregando do arquivo

In [ ]:
def ler_arquivo_qrels_trec20(file_path:str) -> dict:
    """
    Lê um arquivo TSV contendo a avaliação de relevância de documentos para cada consulta.
    
    Args:
    file_path: str - O caminho do arquivo a ser lido.
    
    Returns:
    dict - Um dicionário onde as chaves são os IDs das consultas e os valores são 
           dicionários em que as chaves são os IDs dos documentos e os valores são 
           os níveis de relevância (0, 1, 2, 3, ou 4) de cada documento para a consulta correspondente.
    """
    qrels_dict = {}

    with open(file_path, 'r') as f:
        # Itera sobre cada linha do arquivo
        for line in f:
            # Separa a linha em seus campos
            query_id, _, doc_id, relevance = line.strip().split()
            query_id = int(query_id)
            doc_id = int(doc_id)
            # Verifica se a consulta já existe no dicionário
            if query_id not in qrels_dict:
                qrels_dict[query_id] = {}
            # Adiciona o ID do documento e seu nível de relevância
            qrels_dict[query_id][doc_id] = int(relevance)
    return qrels_dict

Verificando qrel de todo o dev dataset

In [ ]:
qrel_dev_dict = ler_arquivo_qrels_trec20(f'{path_data}/qrels.dev.small.trec')

In [ ]:
len(qrel_dev_dict),list(qrel_dev_dict.items())[0]

Carregando o qrel do trec20 dataset

In [ ]:
qrel_dev_dict = ler_arquivo_qrels_trec20(f'{path_pyserini_tools}/topics-and-qrels/qrels.dl20-passage.txt')

In [ ]:
len(qrel_dev_dict),list(qrel_dev_dict.items())[0]

Todas as 54 queries possuem informação de relevância

In [ ]:
[query for query, doc_rel in list(qrel_dev_dict.items()) if len(doc_rel)==0]

##### Carregando usando get_qrels

In [ ]:
from pyserini.search import get_qrels

In [ ]:
qrels = get_qrels('dl20-passage')

In [ ]:
len(qrels)

In [ ]:
list(qrels.items())[0]

### Indexando Trec 2020 Collection using Pyserini

In [ ]:
%%time
if not os.path.exists('./indexes/lucene-index-msmarco-passage'):
  !python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input {path_data}/collection_jsonl \
  --index indexes/lucene-index-msmarco-passage \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

In [ ]:
!du -hs './indexes/lucene-index-msmarco-passage'

## Calculando ndcg@10 pelo pyserini no trec 2020 (small dev)

#### Com script

We can also use the official TREC evaluation tool, trec_eval, to compute metrics other than MRR@10. For that we first need to convert the run file into TREC format:



In [ ]:
# trocar abaixo se for para realizar o search todo o dev dataset
# file_topics = 'msmarco-passage-dev-subset'
file_topics_search = 'dl20'
print(f'file_topics_search: {file_topics_search}')

# trocar abaixo se for para realizar o eval todo o dev dataset
#file_topics_eval = {path_data}/qrels.dev.small.trec
file_topics_eval = 'dl20-passage'
print(f'file_topics_eval: {file_topics_eval}')

In [ ]:
num_max_hits = 100

In [ ]:
%%time
!python -m pyserini.search.lucene \
  --index indexes/lucene-index-msmarco-passage \
  --topics {file_topics_search} \
  --output runs/run.msmarco-passage.bm25.trec \
  --output-format msmarco \
  --hits {num_max_hits} \
  --bm25 --k1 0.82 --b 0.68

Here, we set the BM25 parameters to k1=0.82, b=0.68 (tuned by grid search). The option --output-format msmarco says to generate output in the MS MARCO output format. The option --hits specifies the number of documents to return per query. Thus, the output file should have approximately 6980 × num_max_hits (698.000, if it is 100) lines.

Retrieval speed will vary by hardware: On a reasonably modern CPU with an SSD, we might get around 13 qps (queries per second), and so the entire run should finish in under ten minutes (using a single thread). We can perform multi-threaded retrieval by using the --threads and --batch-size arguments. For example, setting --threads 16 --batch-size 64 on a CPU with sufficient cores, the entire run will finish in a couple of minutes.

Usamos parâmetro -l 2 seguindo orientação em pyserini\docs\experiments-msmarco-irst.md

(...)
Similarly, for TREC DL 2020:

```bash
python -m pyserini.eval.trec_eval -c -m map -m ndcg_cut.10 -l 2 \
  dl20-passage runs/run.irst-sum.passage.dl20.txt
```


In [ ]:
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 -mrecall.100 -mmap -l 2 {file_topics_eval} runs/run.dl20-passage.bm25.trec

In [ ]:
!python -m pyserini.search.lucene \
  --index indexes/lucene-index-msmarco-passage \
  --topics {file_topics} \
  --output runs/run.dl20-passage.bm25.trec \
  --output-format msmarco \
  --hits {num_max_hits} \
  --bm25 --k1 0.82 --b 0.68

In [ ]:
mostra_memoria()

#### Com código

Adaptado do [caderno do colega Gustavo Bartz Guedes](https://colab.research.google.com/drive/10z86PObSxqbXczZ9pz0T-QurL21oMShf?usp=sharing)

In [ ]:
# code from https://colab.research.google.com/github/castorini/anserini-notebooks/blob/master/pyserini_msmarco_passage_demo.ipynb
from pyserini.search import SimpleSearcher
from pyserini.search.lucene import LuceneSearcher
from tqdm import tqdm


In [ ]:
# Run all queries in topics, retrive top 1k for each query
def run_all_queries(file, topics, searcher, num_max_hits=100):
    with open(file, 'w') as runfile:
        cnt = 0
        print('Running {} queries in total'.format(len(topics)))
        for id in tqdm(topics, desc='Running Queries'):
            query = topics[id]['title']
            hits = searcher.search(query, num_max_hits)
            for i in range(0, len(hits)):
                _ = runfile.write('{} Q0 {} {} {:.6f} Pyserini\n'.format(id, hits[i].docid, i+1, hits[i].score))
            cnt += 1
            if cnt % 100 == 0:
                print(f'{cnt} queries completed')


In [ ]:
searcher = LuceneSearcher('./indexes/lucene-index-msmarco-passage')
searcher.set_bm25(k1=0.82, b=0.68)


In [ ]:
run_all_queries('run-msmarco-passage-bm25.txt', topics, searcher, num_max_hits)

In [ ]:
!head run-msmarco-passage-bm25.txt

##### Eval

In [ ]:
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 -mrecall.100 -mmap -l 2 {file_topics_eval} run-msmarco-passage-bm25.txt

## Pré-processar as queries e os documentos

[Seguindo padrão do lucene](docs/usage-analyzer.md) (Analyzer API)

In [ ]:
from pyserini.analysis import Analyzer, get_lucene_analyzer

In [ ]:
# Default analyzer for English uses the Porter stemmer:
analyzer = Analyzer(get_lucene_analyzer())
tokens = analyzer.analyze('City buses are running on time.')
print(tokens)
# Result is ['citi', 'buse', 'run', 'time']

In [ ]:
assert len(qrels)==54, "qrels não carregado com relevância de 54 queries"

In [ ]:
assert len(topics)==200, "topics não carregado com 200 queries"

In [ ]:
list(topics.items())[0]

Para economizar esforço, retiraremos de topic as queries que não possuem informação de relevância, que não estão em qrel.

In [ ]:
topics_com_relevancia = {key:value for key, value in topics.items() if key in qrels}

In [ ]:
len(topics_com_relevancia)

### Preprocessar documentos da coleção

In [ ]:
!head {path_data}/collection_jsonl/docs08.json

In [ ]:
import json
import os
from pathlib import Path
from typing import List

In [ ]:
# Define a função para pré-processar os documentos
def preprocessar(text: str) -> List[str]:
    # Aqui entra o código para pré-processar o texto
    return analyzer.analyze(text)

In [ ]:
# Define o caminho para a pasta com os arquivos JSON
path_json_passage = f'{path_data}/collection_jsonl'

In [ ]:
def preprocessa_documentos(path_json_passage: str):
  """
  Lê arquivos json no diretório `path_json_passage`, pré-processa os conteúdos (chave 'contents')
  utilizando a função `preprocessar` e salva os novos arquivos com o mesmo nome, mas com sufixo "_prep".

  Args:
    path_json_passage (str): Caminho para o diretório contendo arquivos json.

  """
  # Iterar sobre todos os arquivos no diretório
  for file_name in tqdm(os.listdir(path_json_passage), desc=f'iterando arquivos json em  {path_json_passage}'):
    if file_name.endswith('_prep.json'):
        continue

    print(f'Processando arquivo {file_name}')
    
    # Abrir o arquivo atual para leitura
    file_path = os.path.join(path_json_passage, file_name)
    print('em preprocess_document_lines', file_path)
    with open(file_path, 'r') as f:
        docs_json = {}
        # Ler cada linha do arquivo (que contém um json)
        for line in tqdm(f, desc=f'acessando {file_path}', total=1000000, miniters=100000):
          doc = json.loads(line)
          # Adicionar id do documento e seus tokens pré-processados no dicionário
          docs_json[int(doc['id'])] = preprocessar(doc['contents'])

    # Salvar arquivo pré-processado com novo nome
    new_file_name = os.path.splitext(file_name)[0] + '_prep.json'
    print(f'Gravando arquivo {new_file_name}')
    with open(os.path.join(path_json_passage, new_file_name), 'w') as f:
        json.dump(docs_json, f)


In [ ]:
preprocessa_documentos(path_json_passage)

In [ ]:
mostra_memoria()

In [ ]:
def concatena_jsons(path):
    dict_trec2020 = {}
    for file_name in os.listdir(path):
        if file_name.endswith('_prep.json'):
            print(f'processando {file_name} ')
            with open(os.path.join(path, file_name), 'r') as f:
                dict_trec2020.update(json.load(f))
    return dict_trec2020

In [ ]:
doctos_trec2020_dict = concatena_jsons(path_json_passage)

In [ ]:
mostra_memoria()

In [ ]:
def preprocessa_queries(topics: dict) -> dict:
    """
    Função que pré-processa o texto dos títulos dos tópicos do dataset TREC.

    Args:
        topics (dict): Dicionário com as queries do dataset TREC.

    Returns:
        dict: Dicionário com as queries pré-processadas e seus respectivos tokens.
    """
    topics_prep = {}

    # Itera sobre as queries do dicionário topics
    for query_id, query_text in tqdm(topics.items(), desc="Preprocessando queries"):
        # Realiza o pré-processamento do texto do título
        title_prep = preprocessar(query_text['title'])
        
        # Adiciona o texto pré-processado na chave 'tokens'
        query_text['tokens'] = title_prep
        
        # Adiciona o resultado ao novo dicionário
        topics_prep[query_id] = query_text
    
    return topics_prep

In [ ]:
topics_com_relevancia_prep = preprocessa_queries(topics_com_relevancia)

In [ ]:
len(topics_com_relevancia_prep), list(topics_com_relevancia_prep.items())[0]

## Desenvolvimento dos buscadores

### Criando massa fictícia para testar código

In [ ]:
# Inicializando queries preprocessadas

# Inicializando documentos preprocessados
documentos_prep_test = {
    1: {'title': 'Document about international organized crime.', 'tokens': ['document', 'international', 'organized', 'crime']},
    2: {'title': 'Media violence can increase aggressive behavior.', 'tokens': ['media', 'violence', 'increase', 'aggressive', 'behavior']},
    3: {'title': 'Effective water management strategies.', 'tokens': ['effective', 'water', 'management', 'strategies']},
    4: {'title': 'Report on transnational crime.', 'tokens': ['report', 'transnational', 'crime']},
    5: {'title': 'The role of mass media in shaping public opinion.', 'tokens': ['role', 'mass', 'media', 'shaping', 'public', 'opinion']},
    6: {'title': 'Water scarcity and conflicts.', 'tokens': ['water', 'scarcity', 'conflicts']},
    7: {'title': 'Organized crime in Latin America.', 'tokens': ['organized', 'crime', 'latin', 'america']},
    8: {'title': 'Impact of violent media on youth.', 'tokens': ['impact', 'violent', 'media', 'youth']},
    9: {'title': 'Water resources in the Middle East.', 'tokens': ['water', 'resources', 'middle', 'east']},
    10: {'title': 'Overview of organized crime.', 'tokens': ['overview', 'organized', 'crime']}
}


topics_prep_test = {
    301: {'title': 'International Organized Crime', 'tokens': ['international', 'organized', 'crime']},
    302: {'title': 'Mass Media and Violence', 'tokens': ['mass', 'media' ,'violence']},
    303: {'title': 'Water Management', 'tokens': ['water', 'management']}
}


# Inicializando qrels
qrels_test: Dict[str, Dict[int, int]] = {
    301: {1: 3, 2: 2, 3: 0, 4: 1, 5: 0, 6: 0, 7: 1, 8: 0, 9: 0, 10: 1},
    302: {1: 0, 2: 3, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0},
    303: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 3, 7: 0, 8: 0, 9: 2, 10: 0}
}

### BooleanSearcher


In [ ]:
from collections import Counter
import torch

In [ ]:
class BagofWordsSearcher:
    """
    Classe responsável por criar um índice invertido de tokens de um conjunto de documentos
    e realizar busca baseada na similaridade entre o índice e uma consulta de busca.

    Parâmetros
    ----------
    docs : dict
        Um dicionário onde as chaves são identificadores únicos para cada documento e os valores
        são outros dicionários contendo informações sobre os documentos, como tokens e outras
        informações relevantes.
    device : torch.device, opcional
        O dispositivo (CPU ou GPU) em que o índice invertido e os tensores relacionados serão alocados.
        O valor padrão é "cuda" se o PyTorch detectar que uma GPU está disponível e "cpu" caso contrário.
    parm_se_imprime : bool, opcional
        Um parâmetro para controlar se mensagens de depuração devem ser impressas durante a execução
        da classe. O valor padrão é False, ou seja, as mensagens não serão impressas.

    Atributos
    ---------
    vocab : list
        Uma lista contendo todos os tokens únicos encontrados em todos os documentos.
    docs : dict
        O mesmo dicionário passado como entrada no construtor.
    _device : torch.device
        O dispositivo em que o índice invertido e os tensores relacionados serão alocados.
    _doc_ids : list
        Uma lista de identificadores únicos de documentos, na mesma ordem que o tensor "index".
    _tamanho_vocab : int
        O número total de tokens únicos encontrados em todos os documentos.
    index : torch.Tensor
        Um tensor de tamanho (num_docs, num_tokens), onde cada linha representa um documento e cada
        coluna representa um token, indicando quantas vezes o token aparece no documento.

    Métodos
    -------
    _create_index()
        Cria o índice invertido e armazena os resultados em "vocab", "_doc_ids", "_tamanho_vocab" e
        "index".
    _numericaliza(tokens)
        Converte uma lista de tokens em um tensor representando a contagem de ocorrências de cada
        token na lista, em relação ao vocabulário geral.
    search(query)
        Realiza uma busca baseada na similaridade entre o tensor de contagem de tokens da consulta e
        o tensor de contagem de tokens de todos os documentos. Retorna uma lista de tuplas contendo
        o identificador de cada documento e a medida de similaridade entre a consulta e o documento,
        em ordem decrescente de similaridade.

    Exemplos
    --------
    >>> docs = {
    ...     "doc1": {"tokens": ["foo", "bar", "baz"]},
    ...     "doc2": {"tokens": ["foo", "foo", "bar", "qux"]},
    ...     "doc3": {"tokens": ["baz", "qux", "quux"]}
    ... }
    >>> bws = BagofWordsSearcher(docs)
    >>> bws.search(["foo", "bar"])
    [("doc2", 2.0), ("doc1", 1.0), ("doc3", 0.0)]
    """

    def __init__(self, docs, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'), parm_se_imprime:bool=False):
        """
        Construtor da classe BagofWordsSearcher.

        Args:
            docs (dict): Um dicionário contendo os documentos a serem indexados.
            device (torch.device, optional): Dispositivo onde o índice será armazenado (GPU ou CPU).
                                              O padrão é 'cuda' se uma GPU estiver disponível, caso contrário 'cpu'.
            parm_se_imprime (bool, optional): Indica se informações de depuração devem ser impressas durante a execução.
                                              O padrão é True.

        Attributes:
            _se_imprime (bool): Indica se informações de depuração devem ser impressas durante a execução.
            vocab (list): Lista de palavras únicas encontradas nos documentos.
            docs (dict): Dicionário contendo os documentos a serem indexados.
            _device (torch.device): Dispositivo onde o índice será armazenado (GPU ou CPU).
            _doc_ids (list): Lista com os IDs dos documentos.
            _tamanho_vocab (int): Quantidade de palavras únicas encontradas nos documentos.
            index (torch.Tensor): Matriz onde cada linha representa um documento e cada coluna representa a contagem
                                  de uma palavra única.
        """
        self._se_imprime = parm_se_imprime
        self.vocab = None
        self.docs = docs
        self._device = device

        # Imprime informações de depuração, se necessário
        if self._se_imprime:
            print(f"Em __init__: self._device = {self._device}")
            print(f"Em __init__: len(self.docs) = {len(self.docs)}")

        # Cria o índice invertido que representa todos os documentos da classe em um espaço vetorial.
        self._create_index()

    def _create_index(self):
        """
        Cria o índice invertido que representa todos os documentos da classe em um espaço vetorial.

        Cada documento é convertido em um vetor de tokens e, em seguida, um vocabulário é criado a partir de todos os
        tokens de todos os documentos, sem repetições. A lista de documentos é transformada em uma matriz, onde cada
        linha representa um documento e cada coluna representa um token do vocabulário. Cada posição da matriz representa
        a frequência de um token em um documento.

        Essa matriz é criada no dispositivo definido em self._device.

        """
        # cria o conjunto de vocabulário que representa todos os tokens de todos os documentos, sem repetições
        vocab = set()

        # cria uma lista que vai conter o id de cada documento
        doc_ids = []

        # itera por todos os documentos e atualiza vocab com os tokens de cada documento, e doc_ids com o id do documento
        for doc_id, doc in self.docs.items():

            if type(doc) == dict:
                vocab.update(set(doc['tokens']))
            else: # type(doc) == list
                vocab.update(set(doc))


            doc_ids.append(doc_id)
       

        self.tipo_origem = type(self.docs[doc_ids[0]])

        # transforma o conjunto vocab em uma lista, para preservar a ordem dos tokens
        self.vocab = list(vocab)

        # salva a lista de ids dos documentos
        self._doc_ids = doc_ids

        # salva o tamanho do vocabulário
        self._tamanho_vocab = len(self.vocab)

        # cria a matriz index, onde cada linha representa um documento e cada coluna representa um token do vocabulário
        # a posição (i,j) da matriz representa a frequência do token j no documento i
        # a matriz é criada no dispositivo definido em self._device
        if self.tipo_origem == dict:
            self.index = torch.stack([self._numericaliza(doc["tokens"]) for doc in self.docs.values()]).to(self._device)
        elif self.tipo_origem == list:
            self.index = torch.stack([self._numericaliza(doc) for doc in self.docs.values()]).to(self._device)

        if self._se_imprime:
            print(f"Em _create_index: self.vocab = {self.vocab}")        
            print(f"Em _create_index: self._doc_ids = {self._doc_ids}")        
            print(f"Em _create_index: self._tamanho_vocab = {self._tamanho_vocab}")          
            print(f"Em _create_index: self.index = {self.index}")          
            print(f"Em _create_index: self.tipo_origem = {self.tipo_origem}")          

    def _numericaliza(self, tokens):
        """
        Transforma uma lista de tokens em um tensor com a contagem de ocorrências de cada token na lista.

        Args:
            tokens (list): lista de tokens.

        Returns:
            torch.Tensor: tensor com a contagem de ocorrências de cada token na lista.
        """
        # Cria um objeto Counter com a contagem de ocorrências de cada token na lista
        token_counts = Counter(tokens)
        
        # Obtém os índices de cada token na lista de vocabulário (se existir)
        indexes = [self.vocab.index(token) for token in token_counts.keys() if token in self.vocab]
        
        # Cria um tensor com zeros, com o mesmo tamanho do vocabulário
        values = torch.zeros(self._tamanho_vocab, device=self._device)
        
        # Para cada token na contagem, atualiza o tensor values na posição correspondente ao índice do token
        for token, count in token_counts.items():
            if token in self.vocab:
                values[self.vocab.index(token)] = count

        if self._se_imprime:
            print(f"Em _numericaliza: token_counts = {token_counts}")
            print(f"Em _numericaliza: indexes = {indexes}")
            print(f"Em _numericaliza: values = {values}")

        return values


    def search(self, query: list, k:int=10):
        """
        Realiza uma busca por similaridade entre o documento e a query fornecidos. Retorna uma lista de tuplas
        contendo o id do documento e sua similaridade com a query, ordenada de forma decrescente pela similaridade.

        Parâmetros:
        -----------
        query : list
            Lista de tokens da query.

        Retorno:
        --------
        relevant_docs : list
            Lista de tuplas (id do documento, similaridade) ordenada de forma decrescente pela similaridade.
        """
        # Converte a query em um tensor numérico.
        query_tensor = self._numericaliza(query).unsqueeze(0).to(self._device)
                    
        # Calcula a similaridade entre a query e todos os documentos da base de dados.
        similarities = torch.matmul(query_tensor, self.index.T).squeeze(dim=0)
                
        # Gera uma lista de tuplas contendo o id do documento e sua similaridade com a query.
        result = [(self._doc_ids[i], s) for i, s in enumerate(similarities.tolist())]
                    
        # Ordena a lista de documentos relevantes pela similaridade em ordem decrescente.
        relevant_docs = sorted(result, key=lambda x: x[1], reverse=True)[:k]

        if self._se_imprime:
            print(f"Em search: query_tensor = {query_tensor}")
            print(f"Em search: similarities = {similarities}")
            print(f"Em search: result = {result}")
            print(f"Em search: relevant_docs = {relevant_docs}")                    
        return relevant_docs


In [ ]:
bow_searcher = BagofWordsSearcher(documentos_prep_test, parm_se_imprime=True)

In [ ]:
bow_searcher.tipo_origem

In [ ]:
bow_searcher.search(topics_prep_test[301]['tokens'],k=5)

## Calculando a métrica ndcg@10

In [ ]:
bow_searcher = BagofWordsSearcher(documentos_prep_test, parm_se_imprime=False)

In [ ]:
def ndcg_at_k(ranking_docto, relevance_ordenada, dict_relevancia, k=10):
    dcg = 0.0
    idcg = 0.0
    for i, docto_id in enumerate(ranking_docto):
        if i > k:
            break
        dcg += dict_relevancia[docto_id] / torch.log2(torch.tensor(i + 2))
        idcg += dict_relevancia[relevance_ordenada[i]] / torch.log2(torch.tensor(i + 2))
        print(f'i={i}, docto_id={docto_id} dcg={dcg} idcg={idcg}')

    val_metric = dcg / idcg
    print(f"val_metric = dcg / idcg :: {val_metric} = {dcg} / {idcg}  ")
    return dcg / idcg

In [ ]:
import math

In [ ]:
def ndcg_at_k_query(ranking_docto, relevance_ordenada, dict_relevancia, k=10):
    """
    Calcula a métrica NDCG@k.

    Args:
        ranking_docto (list): Lista com o ID dos documentos retornados pela busca.
        relevance_ordenada (list): Lista com o ID dos documentos relevantes para a consulta, 
                                   ordenados pela relevância.
        dict_relevancia (dict): Dicionário que mapeia o ID do documento à sua relevância.
        k (int): Número de documentos considerados na métrica.

    Returns:
        float: O valor da métrica NDCG@k para a consulta.

    """
    dcg = 0.0  # inicializa o valor de dcg como 0
    idcg = 0.0  # inicializa o valor de idcg como 0

    # percorre o ranking de documentos
    for i, docto_id in enumerate(ranking_docto):
        if i >= k:
            break  # para de processar documentos se já chegou no número k

        # calcula o valor de dcg para o documento atual
        rel = dict_relevancia[docto_id]  # relevância do documento

        # calcula o valor de idcg para o documento atual
        rel_idcg = dict_relevancia[relevance_ordenada[i]]  # relevância do documento considerado ideal

        # acumula 
        if rel > 0:
            dcg += (2 ** rel - 1) / math.log2(i + 2)
        if rel_idcg > 0:
            idcg += (2 ** rel_idcg - 1) / math.log2(i + 2)

        # imprime as informações para depuração
        print(f'i={i}, docto_id={docto_id}  rel={rel} rel_idcg={rel_idcg} dcg={round(dcg,2)} idcg={round(idcg,2)}')

    # calcula o valor final da métrica
    val_metric = dcg / idcg if idcg > 0 else 0.0
    print(f"val_metric = dcg / idcg :: {val_metric} = {dcg} / {idcg}  ")
    return round(val_metric,2)

In [ ]:
def calcula_ndcg_at_k(topics, qrels, searcher, k, se_imprime:bool=False):
    ndcg_scores = []
    for query_id, query in topics.items():
        # Realizando a busca
        results = bow_searcher.search(query['tokens'], k=k)

        # obtém as relevâncias para a query atual
        dict_relevancia = qrels[query_id]
        relevances = [id_docto for id_docto, relevance in sorted(dict_relevancia.items(), key=lambda x: x[1], reverse=True)]

        # Obtendo o ranking com o id dos documentos retornados
        ranking = [par_docid_relevance[0] for par_docid_relevance in results]

        # Calculando a métrica ndcg@10
        ndcg_score = ndcg_at_k_query(ranking, relevances, dict_relevancia, k=10)
        if se_imprime:
            print(f"no cálculo da métrica: query_id={query_id}, query['tokens']={query['tokens']}")
            print(f'no cálculo da métrica: results = {results}')
            print(f'no cálculo da métrica: dict_relevancia = {dict_relevancia}')    
            print(f'no cálculo da métrica: relevances = {relevances}')
            print(f'no cálculo da métrica: ranking = {ranking}')
            print(f'no cálculo da métrica: ndcg_score = {ndcg_score}')  
        # Armazenando a métrica para a query atual
        ndcg_scores.append((query_id, ndcg_score))
        
    # Calculando a média dos ndcg
    ndcg_mean = sum([score[1] for score in ndcg_scores])/len(ndcg_scores)
    return ndcg_mean, ndcg_scores

In [ ]:
ndcg_mean, ndcg_scores = calcula_ndcg_at_k(topics_prep_test, qrels_test, bow_searcher, k=10)
print(f"ndcg_mean: {ndcg_mean}")
print(f"ndcg_scores: {ndcg_scores}")

In [ ]:
class BooleanSearcher:
    """
    Classe responsável por criar um índice invertido de tokens de um conjunto de documentos
    e realizar busca baseada na similaridade entre o índice e uma consulta de busca.

    Parâmetros
    ----------
    docs : dict
        Um dicionário onde as chaves são identificadores únicos para cada documento e os valores
        são outros dicionários contendo informações sobre os documentos, como tokens e outras
        informações relevantes.
    device : torch.device, opcional
        O dispositivo (CPU ou GPU) em que o índice invertido e os tensores relacionados serão alocados.
        O valor padrão é "cuda" se o PyTorch detectar que uma GPU está disponível e "cpu" caso contrário.
    parm_se_imprime : bool, opcional
        Um parâmetro para controlar se mensagens de depuração devem ser impressas durante a execução
        da classe. O valor padrão é False, ou seja, as mensagens não serão impressas.

    Atributos
    ---------
    vocab : list
        Uma lista contendo todos os tokens únicos encontrados em todos os documentos.
    docs : dict
        O mesmo dicionário passado como entrada no construtor.
    _device : torch.device
        O dispositivo em que o índice invertido e os tensores relacionados serão alocados.
    _doc_ids : list
        Uma lista de identificadores únicos de documentos, na mesma ordem que o tensor "index".
    _tamanho_vocab : int
        O número total de tokens únicos encontrados em todos os documentos.
    index : torch.Tensor
        Um tensor de tamanho (num_docs, num_tokens), onde cada linha representa um documento e cada
        coluna representa um token, indicando quantas vezes o token aparece no documento.

    Métodos
    -------
    _create_index()
        Cria o índice invertido e armazena os resultados em "vocab", "_doc_ids", "_tamanho_vocab" e
        "index".
    _numericaliza(tokens)
        Converte uma lista de tokens em um tensor representando a contagem de ocorrências de cada
        token na lista, em relação ao vocabulário geral.
    search(query)
        Realiza uma busca baseada na similaridade entre o tensor de contagem de tokens da consulta e
        o tensor de contagem de tokens de todos os documentos. Retorna uma lista de tuplas contendo
        o identificador de cada documento e a medida de similaridade entre a consulta e o documento,
        em ordem decrescente de similaridade.

    Exemplos
    --------
    >>> docs = {
    ...     "doc1": {"tokens": ["foo", "bar", "baz"]},
    ...     "doc2": {"tokens": ["foo", "foo", "bar", "qux"]},
    ...     "doc3": {"tokens": ["baz", "qux", "quux"]}
    ... }
    >>> bws = BagofWordsSearcher(docs)
    >>> bws.search(["foo", "bar"])
    [("doc2", 2.0), ("doc1", 1.0), ("doc3", 0.0)]
    """

    def __init__(self, docs, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'), parm_se_imprime:bool=False):
        """
        Construtor da classe BagofWordsSearcher.

        Args:
            docs (dict): Um dicionário contendo os documentos a serem indexados.
            device (torch.device, optional): Dispositivo onde o índice será armazenado (GPU ou CPU).
                                              O padrão é 'cuda' se uma GPU estiver disponível, caso contrário 'cpu'.
            parm_se_imprime (bool, optional): Indica se informações de depuração devem ser impressas durante a execução.
                                              O padrão é True.

        Attributes:
            _se_imprime (bool): Indica se informações de depuração devem ser impressas durante a execução.
            vocab (list): Lista de palavras únicas encontradas nos documentos.
            docs (dict): Dicionário contendo os documentos a serem indexados.
            _device (torch.device): Dispositivo onde o índice será armazenado (GPU ou CPU).
            _doc_ids (list): Lista com os IDs dos documentos.
            _tamanho_vocab (int): Quantidade de palavras únicas encontradas nos documentos.
            index (torch.Tensor): Matriz onde cada linha representa um documento e cada coluna representa a contagem
                                  de uma palavra única.
        """
        self._se_imprime = parm_se_imprime
        self.vocab = None
        self.docs = docs
        self._device = device

        # Imprime informações de depuração, se necessário
        if self._se_imprime:
            print(f"Em __init__: self._device = {self._device}")
            print(f"Em __init__: len(self.docs) = {len(self.docs)}")

        # Cria o índice invertido que representa todos os documentos da classe em um espaço vetorial.
        self._create_index()

    def _create_index(self):
        """
        Cria o índice invertido que representa todos os documentos da classe em um espaço vetorial.

        Cada documento é convertido em um vetor de tokens e, em seguida, um vocabulário é criado a partir de todos os
        tokens de todos os documentos, sem repetições. A lista de documentos é transformada em uma matriz, onde cada
        linha representa um documento e cada coluna representa um token do vocabulário. Cada posição da matriz representa
        a frequência de um token em um documento.

        Essa matriz é criada no dispositivo definido em self._device.

        """
        # cria o conjunto de vocabulário que representa todos os tokens de todos os documentos, sem repetições
        vocab = set()

        # cria uma lista que vai conter o id de cada documento
        doc_ids = []

        # itera por todos os documentos e atualiza vocab com os tokens de cada documento, e doc_ids com o id do documento
        for doc_id, doc in self.docs.items():

            if type(doc) == dict:
                vocab.update(set(doc['tokens']))
            else: # type(doc) == list
                vocab.update(set(doc))


            doc_ids.append(doc_id)
       

        self.tipo_origem = type(self.docs[doc_ids[0]])

        # transforma o conjunto vocab em uma lista, para preservar a ordem dos tokens
        self.vocab = list(vocab)

        # salva a lista de ids dos documentos
        self._doc_ids = doc_ids

        # salva o tamanho do vocabulário
        self._tamanho_vocab = len(self.vocab)

        # cria a matriz index, onde cada linha representa um documento e cada coluna representa um token do vocabulário
        # a posição (i,j) da matriz representa a frequência do token j no documento i
        # a matriz é criada no dispositivo definido em self._device
        if self.tipo_origem == dict:
            self.index = torch.stack([self._numericaliza(doc["tokens"]) for doc in self.docs.values()]).to(self._device)
        elif self.tipo_origem == list:
            self.index = torch.stack([self._numericaliza(doc) for doc in self.docs.values()]).to(self._device)

        if self._se_imprime:
            print(f"Em _create_index: self.vocab = {self.vocab}")        
            print(f"Em _create_index: self._doc_ids = {self._doc_ids}")        
            print(f"Em _create_index: self._tamanho_vocab = {self._tamanho_vocab}")          
            print(f"Em _create_index: self.index = {self.index}")          
            print(f"Em _create_index: self.tipo_origem = {self.tipo_origem}")          

    def _numericaliza(self, tokens):
        """
        Transforma uma lista de tokens em um tensor com a indicação se ocorre (sim ou não) cada token na lista.

        Args:
            tokens (list): lista de tokens.

        Returns:
            torch.Tensor: tensor com a indicação se ocorre (sim ou não) cada token na lista.
        """
        # Cria um objeto Counter com a contagem de ocorrências de cada token na lista
        token_counts = Counter(tokens)
        
        # Obtém os índices de cada token na lista de vocabulário (se existir)
        indexes = [self.vocab.index(token) for token in token_counts.keys() if token in self.vocab]
        
        # Cria um tensor com zeros, com o mesmo tamanho do vocabulário
        values = torch.zeros(self._tamanho_vocab, device=self._device)
        
        # Para cada token na contagem, atualiza o tensor values na posição correspondente ao índice do token
        for token, count in token_counts.items():
            if token in self.vocab:
                values[self.vocab.index(token)] = 1

        if self._se_imprime:
            print(f"Em _numericaliza: token_counts = {token_counts}")
            print(f"Em _numericaliza: indexes = {indexes}")
            print(f"Em _numericaliza: values = {values}")

        return values


    def search(self, query: list, k:int=10):
        """
        Realiza uma busca por similaridade entre o documento e a query fornecidos. Retorna uma lista de tuplas
        contendo o id do documento e sua similaridade com a query, ordenada de forma decrescente pela similaridade.

        Parâmetros:
        -----------
        query : list
            Lista de tokens da query.

        Retorno:
        --------
        relevant_docs : list
            Lista de tuplas (id do documento, similaridade) ordenada de forma decrescente pela similaridade.
        """
        # Converte a query em um tensor numérico.
        query_tensor = self._numericaliza(query).unsqueeze(0).to(self._device)
                    
        # Calcula a similaridade entre a query e todos os documentos da base de dados.
        similarities = torch.matmul(query_tensor, self.index.T).squeeze(dim=0)
                
        # Gera uma lista de tuplas contendo o id do documento e sua similaridade com a query.
        result = [(self._doc_ids[i], s) for i, s in enumerate(similarities.tolist())]
                    
        # Ordena a lista de documentos relevantes pela similaridade em ordem decrescente.
        relevant_docs = sorted(result, key=lambda x: x[1], reverse=True)[:k]

        if self._se_imprime:
            print(f"Em search: query_tensor = {query_tensor}")
            print(f"Em search: similarities = {similarities}")
            print(f"Em search: result = {result}")
            print(f"Em search: relevant_docs = {relevant_docs}")                    
        return relevant_docs


### Criando searcher para trec2020

Testando em um pedaço do doctos_trec2020_dict

In [ ]:
parte_doctos_trec2020_dict = {key: value for key, value in list(doctos_trec2020_dict.items())[0:2]}

In [ ]:
bow_searcher = BagofWordsSearcher(parte_doctos_trec2020_dict, parm_se_imprime=False)

In [ ]:
topics_com_relevancia_prep[23849]

In [ ]:
bow_searcher.search(topics_com_relevancia_prep[23849])

Criando para todo o doctos_trec2020_dict

In [ ]:
mostra_memoria()

In [ ]:
bow_searcher = BagofWordsSearcher(doctos_trec2020_dict, parm_se_imprime=False)

In [ ]:
mostra_memoria()

In [ ]:
ndcg_mean, ndcg_scores = calcula_ndcg_at_k(topics_com_relevancia_prep, qrels, bow_searcher, k=10)
print(f"ndcg_mean: {ndcg_mean}")
print(f"ndcg_scores: {ndcg_scores}")